<a href="https://colab.research.google.com/github/Greta2093/tribe_of_ai_class_D_project/blob/master/Project_darft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import all the important libraries and matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
main_agrr_total = pd.read_excel('/content/drive/My Drive/Search_Python/main_agrr_total-3.xlsx', index_col=0)
main_agrr_daily = pd.read_excel('/content/drive/My Drive/Search_Python/main_agrr_daily-3.xlsx', index_col=0)

## Most Searched Keywords

From this graph we can see the most popular keywords during the given period. One clear conclusion could be drawn - "keyword 905" is very popular, yet has no Google Search clicks (paid nor unpaid) thus should be covered by the marketing team.

In [ ]:
#Create dataframe with top 10 keywords
main_agrr_total = main_agrr_total.rename(columns={"Search Query Coded": "Search Query"})
main_agrr_total_top_ten = main_agrr_total.sort_values(by=['All Clicks and Searches'], ascending=False).head(10)

#Create figure with two y axes
fig = make_subplots(specs=[[{"secondary_y": True}]])

#Add data
fig.add_trace(go.Bar(name='Total Unique Searches', x=main_agrr_total_top_ten['Search Query'], y=main_agrr_total_top_ten['Total Unique Searches']))
fig.add_trace(go.Bar(name='Organic Search Clicks', x=main_agrr_total_top_ten['Search Query'], y=main_agrr_total_top_ten['Organic Search Clicks']))
fig.add_trace(go.Bar(name='Paid Clicks', x=main_agrr_total_top_ten['Search Query'], y=main_agrr_total_top_ten['Paid Clicks']))
fig.add_trace(go.Scatter(name='Cost', x=main_agrr_total_top_ten['Search Query'], y=main_agrr_total_top_ten['Cost']),secondary_y=True)

#Update axes names
fig.update_yaxes(title_text="Searches/ Clicks", secondary_y=False)
fig.update_yaxes(rangemode="tozero", title_text="Media Spenditure", secondary_y=True)

#Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [ ]:
#Filter data by keyword
kw1_filter =  main_agrr_daily['Search Query Coded']=='keyword 905'
kw1_data = main_agrr_daily[kw1_filter]

#Create a time series and a histogram for unique searches for this specific keyword
fig3 = make_subplots(rows=1, cols=2)

fig3.add_trace(
    go.Bar(x=kw1_data['Date_datetime64'], y=kw1_data['Total Unique Searches'], name='Time Series'),
    row=1, col=1)

fig3.add_trace(
    go.Histogram(x=kw1_data['Total Unique Searches'], opacity=0.75, name='Histogram'),
    row=1, col=2)

### **Weekday**


In [ ]:
#Add a day of week variable, make it categorical and ordered

cats = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

main_agrr_daily['Date_datetime64'] = pd.to_datetime(main_agrr_daily['Date_datetime64'])
main_agrr_daily = main_agrr_daily.set_index('Date_datetime64')

main_agrr_daily['Day of Week'] = main_agrr_daily.index.day_name()
main_agrr_daily['Day of Week'] = pd.Categorical(main_agrr_daily['Day of Week'], categories=
    ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday'],
    ordered=True)

#Create an aggregated data frame for day of week information
main_agrr_dayofweek = main_agrr_daily.groupby(['Day of Week']).agg({
    'Total Unique Searches': 'sum',
    'Organic Search Clicks': 'sum',
    'Paid Clicks': 'sum',
    'Impressions': 'sum',
    'Transactions': 'sum',
    'Revenue': 'sum',
    'Cost': 'sum',
    'Average Position': 'mean'})

#Create bar chart
fig5 = go.Figure(data=[
    go.Scatter(name='Total Unique Searches', x=main_agrr_dayofweek.index, y=main_agrr_dayofweek['Total Unique Searches']),
    go.Scatter(name='Organic Search Clicks', x=main_agrr_dayofweek.index, y=main_agrr_dayofweek['Organic Search Clicks']),
    go.Scatter(name='Paid Clicks', x=main_agrr_dayofweek.index, y=main_agrr_dayofweek['Paid Clicks']),
])

fig5.show()

In [ ]:
#Create variable of two values - weekend or weekday

main_agrr_daily['Weekend/ Weekday Float'] = ((pd.DatetimeIndex(main_agrr_daily.index).dayofweek) // 5 == 1).astype(float)

main_agrr_daily['Weekend/ Weekday Category'] = 'Weekend'
main_agrr_daily.loc[(main_agrr_daily['Weekend/ Weekday Float'] == 0), 'Weekend/ Weekday Category'] = 'Weekday'

#Create pie charts

fig4 = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]], 
                     subplot_titles=['Cost', 'Revenue'])


fig4.add_trace(go.Pie(
     values=main_agrr_daily['Cost'],
     labels=main_agrr_daily['Weekend/ Weekday Category']), 
     row=1, col=1)


fig4.add_trace(go.Pie(
     values=main_agrr_daily['Revenue'],
     labels=main_agrr_daily['Weekend/ Weekday Category']), 
     row=1, col=2)

In [ ]:
#Create a box plot

main_agrr_daily_withRevenue = main_agrr_daily.loc[main_agrr_daily['Revenue'] > 0]

fig6 = px.box(main_agrr_daily_withRevenue, y="Revenue", color="Weekend/ Weekday Category")
fig6.show()